# EE 123 Lab 2 — Spectral Analysis: Images & Audio

### Written by Tingle Li and John Wang, Spring 2026

In this lab we explore spectral analysis in two domains: **images** (2D DFT) and **audio** (1D FFT / STFT). 
You will visualize frequency content, apply filters in the frequency domain, create hybrid images, 
analyze real audio spectrograms, and perform tone removal — all using the Fourier transform as the central tool.

**Prerequisites:** Lab 1 (DTFT, chirps, cross-correlation).

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, figaspect
from scipy import signal
import librosa
import librosa.display
import IPython.display as ipd

%matplotlib inline

---
# Part 1: Image Spectral Analysis

## Task 1.1: 2D DFT Visualization

The 2D Discrete Fourier Transform (DFT) decomposes an image into its spatial frequency components. 
For an $M \times N$ image $f[m,n]$, the 2D DFT is:

$$ F[u,v] = \sum_{m=0}^{M-1} \sum_{n=0}^{N-1} f[m,n] \, e^{-j2\pi(um/M + vn/N)} $$

Low frequencies (near the center after `fftshift`) correspond to smooth regions, 
while high frequencies (near the edges) correspond to sharp transitions like edges and textures.

**Tasks:**
* Load three images: the cameraman, a synthetic checkerboard, and a natural scene (astronaut).
* Compute the 2D FFT of each. Display the **log-magnitude spectrum** and **phase spectrum** side by side.
* Hint: use the `fft2` and `fftshift` functions from `np.fft`. For display, `np.log1p` avoids log(0).

In [ ]:
from skimage import data as skdata

# Load cameraman image
cameraman = plt.imread('pictures/cameraman.png')
if cameraman.ndim == 3:
    cameraman = np.mean(cameraman[:,:,:3], axis=2)
if cameraman.max() > 1.0:
    cameraman = cameraman / 255.0

# Generate a checkerboard pattern (8x8 blocks)
block = 32
checker = np.kron(
    [[1, 0]*4, [0, 1]*4]*4,
    np.ones((block, block))
)

# Natural scene from skimage
astronaut_rgb = skdata.astronaut()
astronaut = np.mean(astronaut_rgb / 255.0, axis=2)

images = {'Cameraman': cameraman, 'Checkerboard': checker, 'Astronaut': astronaut}

# Display original images
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
for ax, (name, img) in zip(axes, images.items()):
    ax.imshow(img, cmap='gray')
    ax.set_title(name)
    ax.axis('off')
plt.suptitle('Original Images', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Compute and display 2D FFT: log-magnitude and phase
# Hint: for each image, compute the 2D FFT, shift the zero-frequency to center,
# then compute log(1 + |F|) for magnitude and the angle for phase.

fig, axes = plt.subplots(3, 3, figsize=(14, 12))

for i, (name, img) in enumerate(images.items()):
    # TODO: Compute 2D FFT and shift
    F_shifted = # TODO
    
    # TODO: Compute log-magnitude and phase
    magnitude = # TODO
    phase = # TODO
    
    # Original
    axes[i, 0].imshow(img, cmap='gray')
    axes[i, 0].set_title(f'{name} — Original')
    axes[i, 0].axis('off')
    
    # Log-magnitude spectrum
    axes[i, 1].imshow(magnitude, cmap='gray')
    axes[i, 1].set_title(f'{name} — Log-Magnitude Spectrum')
    axes[i, 1].axis('off')
    
    # Phase spectrum
    axes[i, 2].imshow(phase, cmap='gray')
    axes[i, 2].set_title(f'{name} — Phase Spectrum')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

**Question:** Compare the spectra of the checkerboard vs. the natural image. 
Why does the checkerboard have energy concentrated along specific axes, 
while the natural image does not?

**Your answer here:**



## Task 1.2: Frequency-Domain Filtering

Instead of convolving in the spatial domain, we can multiply in the frequency domain (convolution theorem). 
A **Gaussian low-pass filter** in the frequency domain is:

$$ H_{LP}[u,v] = e^{-\frac{u^2+v^2}{2\sigma^2}} $$

and the corresponding **high-pass filter** is $ H_{HP} = 1 - H_{LP} $.

**Tasks:**
* Construct a 2D Gaussian low-pass mask in the frequency domain.
* Apply low-pass and high-pass filtering to the cameraman image: FFT → multiply by mask → IFFT.
* Try **3 different sigma values** (e.g., 10, 30, 80). Display a grid of results.

Hint: create a distance matrix $D[u,v] = \sqrt{u^2 + v^2}$ centered at the image center, 
then the Gaussian mask is $e^{-D^2/(2\sigma^2)}$.

In [ ]:
# Frequency-domain Gaussian filtering
img = cameraman
rows, cols = img.shape
crow, ccol = rows // 2, cols // 2

# TODO: Create frequency coordinate grid centered at (crow, ccol)
# Hint: use np.arange and np.meshgrid to build U, V arrays
# then compute D = distance from center
u = # TODO
v = # TODO
V, U = np.meshgrid(v, u)
D = # TODO

# Compute FFT of image (shifted)
F = np.fft.fftshift(np.fft.fft2(img))

sigmas = [10, 30, 80]
fig, axes = plt.subplots(len(sigmas), 3, figsize=(14, 12))

for i, sigma in enumerate(sigmas):
    # TODO: Create Gaussian low-pass and high-pass masks
    H_lp = # TODO
    H_hp = # TODO
    
    # TODO: Apply filters in frequency domain and transform back
    # Hint: multiply F by mask, then ifftshift + ifft2, take real part
    img_lp = # TODO
    img_hp = # TODO
    
    # Display
    axes[i, 0].imshow(img, cmap='gray')
    axes[i, 0].set_title(f'Original')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(img_lp, cmap='gray')
    axes[i, 1].set_title(f'Low-Pass (σ={sigma})')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(img_hp, cmap='gray')
    axes[i, 2].set_title(f'High-Pass (σ={sigma})')
    axes[i, 2].axis('off')

plt.suptitle('Frequency-Domain Gaussian Filtering', fontsize=14)
plt.tight_layout()
plt.show()

**Question:** You may notice ringing artifacts when using a binary (ideal) circular mask instead 
of a Gaussian. What causes the ringing, and why does the Gaussian mask reduce it?

**Your answer here:**



## Task 1.3: Hybrid Images

A hybrid image combines the low-frequency content of one image with the high-frequency content 
of another. When viewed up close, the high-frequency image dominates perception; from far away 
(or when squinting), only the low-frequency image is visible. This directly demonstrates how the 
human visual system processes spatial frequencies.

Here we perform the entire operation in the **Fourier domain** — no spatial convolution needed. 
This is the convolution theorem in action:

$$ \text{Hybrid} = \mathcal{F}^{-1}\left[ F_A \cdot H_{LP} + F_B \cdot H_{HP} \right] $$

**Tasks:**
* Load the apple and orange images (grayscale).
* Low-pass filter image A in the frequency domain: FFT → multiply by Gaussian mask → IFFT.
* High-pass filter image B: FFT → multiply by (1 − Gaussian mask) → IFFT.
* Add the two results to create the hybrid.
* Experiment with 3 different sigma values. Display results and their Fourier transforms.

In [ ]:
# Load apple and orange (color)
apple_rgb = plt.imread('pictures/apple.jpeg')
orange_rgb = plt.imread('pictures/orange.jpeg')

# Normalize to [0, 1] — keep all 3 color channels
apple = apple_rgb / 255.0 if apple_rgb.max() > 1 else apple_rgb.astype(np.float64)
orange = orange_rgb / 255.0 if orange_rgb.max() > 1 else orange_rgb.astype(np.float64)

# Match sizes
H = min(apple.shape[0], orange.shape[0])
W = min(apple.shape[1], orange.shape[1])
apple = apple[:H, :W]
orange = orange[:H, :W]

print(f'Image size: {H} x {W} x 3 (RGB)')

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(apple); axes[0].set_title('Apple'); axes[0].axis('off')
axes[1].imshow(orange); axes[1].set_title('Orange'); axes[1].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Hybrid image creation in the frequency domain (color — apply per channel)
rows, cols = apple.shape[:2]
crow, ccol = rows // 2, cols // 2

# Reuse the distance matrix approach from Task 1.2
u = np.arange(rows) - crow
v = np.arange(cols) - ccol
V, U = np.meshgrid(v, u)
D = np.sqrt(U**2 + V**2)

sigmas = [5, 15, 30]

fig, axes = plt.subplots(len(sigmas), 4, figsize=(16, 3 * len(sigmas)))

for i, sigma in enumerate(sigmas):
    # TODO: Create Gaussian low-pass and high-pass masks (same as Task 1.2)
    H_lp = # TODO
    H_hp = # TODO
    
    # TODO: Apply filtering per RGB channel
    # Hint: loop over channels (range(3)), compute FFT of each channel,
    # multiply by the mask, IFFT back, and store in a 3-channel result array
    apple_lp = np.zeros_like(apple)
    orange_hp = np.zeros_like(orange)
    for c in range(3):
        F_apple_c = # TODO: FFT of apple[:, :, c], shifted
        F_orange_c = # TODO: FFT of orange[:, :, c], shifted
        apple_lp[:, :, c] = # TODO: IFFT of filtered apple channel
        orange_hp[:, :, c] = # TODO: IFFT of filtered orange channel
    
    # TODO: Combine and clip
    hybrid = # TODO
    
    # Display
    axes[i, 0].imshow(np.clip(apple_lp, 0, 1))
    axes[i, 0].set_title(f'Apple LP (σ={sigma})')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(np.clip(orange_hp + 0.5, 0, 1))  # shift for visibility
    axes[i, 1].set_title(f'Orange HP (σ={sigma})')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(hybrid)
    axes[i, 2].set_title(f'Hybrid (σ={sigma})')
    axes[i, 2].axis('off')
    
    # Fourier transform of hybrid (show one channel as representative)
    F_hybrid = np.fft.fftshift(np.fft.fft2(hybrid[:, :, 1]))
    axes[i, 3].imshow(np.log1p(np.abs(F_hybrid)), cmap='gray')
    axes[i, 3].set_title(f'Hybrid Spectrum (σ={sigma})')
    axes[i, 3].axis('off')

plt.suptitle('Hybrid Images at Different Cutoff Frequencies', fontsize=14)
plt.tight_layout()
plt.show()

Now let's visualize the Fourier transforms at each stage for your best sigma.
* **Display the log-magnitude spectrum of:** original apple, filtered apple (LP), 
  original orange, filtered orange (HP), and the hybrid.

In [ ]:
# Detailed Fourier analysis for your chosen sigma
# Use one channel (e.g., green) to show the spectra at each stage
# TODO: Pick the sigma that works best from the experiment above
sigma = # TODO

H_lp = np.exp(-D**2 / (2 * sigma**2))
H_hp = 1 - H_lp

# TODO: Compute spectra of the green channel (channel index 1) for visualization
F_apple_g = # TODO: FFT of apple[:, :, 1], shifted
F_apple_lp_g = # TODO: apply low-pass mask
F_orange_g = # TODO: FFT of orange[:, :, 1], shifted
F_orange_hp_g = # TODO: apply high-pass mask
F_hybrid_g = F_apple_lp_g + F_orange_hp_g

# Display all spectra
spectra = {
    'Apple (original)': F_apple_g,
    'Apple (low-pass)': F_apple_lp_g,
    'Orange (original)': F_orange_g,
    'Orange (high-pass)': F_orange_hp_g,
    'Hybrid': F_hybrid_g,
}

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for ax, (name, spec) in zip(axes, spectra.items()):
    ax.imshow(np.log1p(np.abs(spec)), cmap='gray')
    ax.set_title(name)
    ax.axis('off')

plt.suptitle(f'Log-Magnitude Spectra at Each Stage (σ={sigma})', fontsize=14)
plt.tight_layout()
plt.show()

**Question:** If you squint or shrink the hybrid image to a thumbnail, which image dominates 
and why? Relate your answer to how the human visual system processes spatial frequencies.

**Your answer here:**



## Task 1.4: Phase vs. Magnitude — Which Carries More Information?

A Fourier transform $F[u,v]$ is fully described by its **magnitude** $|F[u,v]|$ and **phase** $\angle F[u,v]$:

$$ F[u,v] = |F[u,v]| \, e^{j\angle F[u,v]} $$

A classic experiment reveals which component matters more for visual perception: take the magnitude spectrum from one image and combine it with the phase spectrum of another, then reconstruct via inverse FFT.

**Your task (minimal scaffolding — design the experiment yourself):**
* Use two of the grayscale images from Task 1.1 (e.g., `cameraman` and `astronaut`).
* Compute the 2D FFT of each image.
* Construct two "swapped" images:
  - **Reconstruction 1:** magnitude of image A + phase of image B
  - **Reconstruction 2:** magnitude of image B + phase of image A
* Reconstruct via inverse FFT and display all four images (2 originals + 2 reconstructions) in a 2×2 grid.

Useful functions: `np.fft.fft2`, `np.fft.ifft2`, `np.abs`, `np.angle`, `np.exp`.

In [ ]:
# Phase vs. Magnitude reconstruction
# Images from Task 1.1 are already loaded: cameraman, astronaut
# Note: the two images may have different sizes — crop to the smaller dimensions first.

# TODO: Crop both images to matching dimensions
h = min(cameraman.shape[0], astronaut.shape[0])
w = min(cameraman.shape[1], astronaut.shape[1])
img_A = cameraman[:h, :w]
img_B = astronaut[:h, :w]

# TODO: Compute 2D FFT of both images
F_A = # TODO
F_B = # TODO

# TODO: Extract magnitude and phase from each
mag_A = # TODO
phase_A = # TODO
mag_B = # TODO
phase_B = # TODO

# TODO: Combine — mag(A) + phase(B) and mag(B) + phase(A)
# Hint: reconstruct F from magnitude and phase via  mag * exp(1j * phase)
recon1 = # TODO
recon2 = # TODO

# Display
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes[0, 0].imshow(img_A, cmap='gray')
axes[0, 0].set_title('Cameraman (original)')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_B, cmap='gray')
axes[0, 1].set_title('Astronaut (original)')
axes[0, 1].axis('off')

axes[1, 0].imshow(recon1, cmap='gray')
axes[1, 0].set_title('Mag(Cameraman) + Phase(Astronaut)')
axes[1, 0].axis('off')

axes[1, 1].imshow(recon2, cmap='gray')
axes[1, 1].set_title('Mag(Astronaut) + Phase(Cameraman)')
axes[1, 1].axis('off')

plt.suptitle('Phase vs. Magnitude Reconstruction', fontsize=14)
plt.tight_layout()
plt.show()


**Question:** Which image does each reconstruction resemble more — the one that contributed 
its magnitude, or its phase? What does this tell you about the relative importance of 
phase vs. magnitude for human visual perception?

**Your answer here:**



## Task 1.5: Conceptual Wrap-Up

**Question:** In your own words, explain the relationship between spatial features 
(edges, textures, smooth regions) and their frequency-domain representation.

**Your answer here:**



---
# Part 2: Audio Spectral Analysis

## Task 2.1: FFT of Synthetic Signals

We start with synthetic signals where the ground truth is known. Given a signal composed of sinusoids:

$$ x[n] = \sum_{k} A_k \sin(2\pi f_k n / f_s) $$

the FFT should reveal peaks at exactly $f_1, f_2, \ldots$ We will also explore two important 
practical effects: **zero-padding** and **windowing**.

**Tasks:**
* Generate a signal with 3 sinusoids at 440 Hz, 1000 Hz, and 2500 Hz (sampled at 16 kHz, 0.5 s).
* Compute and plot the FFT magnitude spectrum.
* Explore zero-padding: compute FFT with N, 2N, 4N points.
* Explore windowing: compare rectangular vs. Hann window.

In [ ]:
# Generate synthetic signal: sum of 3 sinusoids
fs_audio = 16000
T_audio = 0.5  # seconds
t_audio = np.arange(0, T_audio, 1.0 / fs_audio)
N_audio = len(t_audio)

f1, f2, f3 = 440, 1000, 2500

# TODO: Generate the signal as a sum of 3 sinusoids
# with amplitudes 1.0, 0.7, and 0.5 respectively
x = # TODO

# TODO: Compute FFT and frequency axis
# Hint: use np.fft.fft and np.fft.fftfreq
X = # TODO
freqs = # TODO

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Time domain
axes[0].plot(t_audio[:500], x[:500])
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Synthetic Signal (first 500 samples)')
axes[0].grid(True)

# Frequency domain (positive frequencies only)
pos_mask = freqs >= 0
axes[1].plot(freqs[pos_mask], np.abs(X[pos_mask]))
axes[1].set_xlabel('Frequency [Hz]')
axes[1].set_ylabel('|X(f)|')
axes[1].set_title('Magnitude Spectrum')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Zero-padding exploration
# Hint: pass the n parameter to np.fft.fft to control the FFT length

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for i, nfft in enumerate([N_audio, 2 * N_audio, 4 * N_audio]):
    # TODO: Compute zero-padded FFT
    X_zp = # TODO
    freqs_zp = # TODO
    pos_mask = freqs_zp >= 0
    
    axes[i].plot(freqs_zp[pos_mask], np.abs(X_zp[pos_mask]))
    axes[i].set_xlabel('Frequency [Hz]')
    axes[i].set_ylabel('|X(f)|')
    axes[i].set_title(f'FFT with {nfft} points ({nfft // N_audio}x zero-padded)')
    axes[i].set_xlim([0, 3000])
    axes[i].grid(True)

plt.suptitle('Effect of Zero-Padding on FFT', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Windowing exploration: rectangular vs. Hann
# Hint: np.hanning(N) creates a Hann window of length N.
# Multiplying the signal by the window before FFT reduces spectral leakage.

# TODO: Create Hann window and apply to signal
window_hann = # TODO
x_hann = # TODO

nfft = 4 * N_audio  # zero-pad for smoother display

# TODO: Compute FFTs of rectangular-windowed and Hann-windowed signals
X_rect = # TODO
X_hann = # TODO
freqs_w = np.fft.fftfreq(nfft, 1.0 / fs_audio)
pos_mask = freqs_w >= 0

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Linear scale
axes[0].plot(freqs_w[pos_mask], np.abs(X_rect[pos_mask]), label='Rectangular', alpha=0.7)
axes[0].plot(freqs_w[pos_mask], np.abs(X_hann[pos_mask]), label='Hann', alpha=0.7)
axes[0].set_xlabel('Frequency [Hz]')
axes[0].set_ylabel('|X(f)|')
axes[0].set_title('Windowing Effect (Linear Scale)')
axes[0].set_xlim([0, 3000])
axes[0].legend()
axes[0].grid(True)

# dB scale
X_rect_db = 20 * np.log10(np.abs(X_rect[pos_mask]) + 1e-12)
X_hann_db = 20 * np.log10(np.abs(X_hann[pos_mask]) + 1e-12)
axes[1].plot(freqs_w[pos_mask], X_rect_db, label='Rectangular', alpha=0.7)
axes[1].plot(freqs_w[pos_mask], X_hann_db, label='Hann', alpha=0.7)
axes[1].set_xlabel('Frequency [Hz]')
axes[1].set_ylabel('Magnitude [dB]')
axes[1].set_title('Windowing Effect (dB Scale)')
axes[1].set_xlim([0, 3000])
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

**Question:** Does zero-padding actually improve frequency resolution, or does it just 
interpolate the existing spectrum? Explain, and relate back to the DTFT window analysis from Lab 1.

**Your answer here:**



## Task 2.2: STFT & Spectrogram — Time-Frequency Tradeoff

A single FFT gives us frequency content but loses all time information. The **Short-Time Fourier 
Transform (STFT)** recovers time localization by applying the FFT to short, overlapping windows:

$$ \text{STFT}\{x\}(m, \omega) = \sum_{n} x[n] \, w[n - m] \, e^{-j\omega n} $$

There is a fundamental tradeoff:
* **Short window** → good time resolution, poor frequency resolution
* **Long window** → good frequency resolution, poor time resolution

**Tasks:**
* Generate a synthetic chirp (200 Hz to 4000 Hz, 2 seconds, sampled at 16 kHz).
* Compute spectrograms using `librosa.stft` with 3 different `n_fft` values (256, 1024, 4096).
* Display and compare. Use `hop_length = n_fft // 4`.

In [ ]:
# Generate a synthetic chirp
fs_chirp = 16000
T_chirp = 2.0
t_chirp = np.linspace(0, T_chirp, int(fs_chirp * T_chirp), endpoint=False)

f0_chirp = 200
f1_chirp = 4000

# TODO: Generate the chirp signal
# Recall from Lab 1: for a linear chirp, x(t) = sin(2π(f0 + k/2 * t) * t)
# where k = (f1 - f0) / T is the chirp rate
chirp = # TODO
chirp = chirp.astype(np.float32)

# Play the chirp
ipd.display(ipd.Audio(chirp, rate=fs_chirp))

In [ ]:
# Spectrograms with different window sizes
n_ffts = [256, 1024, 4096]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, n_fft in enumerate(n_ffts):
    hop_length = n_fft // 4
    
    # TODO: Compute STFT and convert to dB
    # Hint: use librosa.stft, then take abs and convert with librosa.amplitude_to_db
    S = # TODO
    S_db = # TODO
    
    librosa.display.specshow(S_db, sr=fs_chirp, hop_length=hop_length,
                             x_axis='time', y_axis='hz', ax=axes[i])
    axes[i].set_title(f'n_fft = {n_fft}\n'
                      f'Time res: {hop_length/fs_chirp*1000:.1f} ms, '
                      f'Freq res: {fs_chirp/n_fft:.1f} Hz')
    axes[i].set_ylim([0, 5000])

plt.suptitle('Time-Frequency Tradeoff: Window Size Effect on Spectrogram', fontsize=14)
plt.tight_layout()
plt.show()

**Question:** Why can't you have perfect resolution in both time and frequency simultaneously? 
Which window size gives the 'best' spectrogram for this chirp, and why?

**Your answer here:**



## Task 2.3: Real Audio Spectrogram Analysis

Now we apply spectrogram analysis to real audio. Musical instruments produce **harmonic** signals: 
a fundamental frequency $f_0$ plus overtones at integer multiples $2f_0, 3f_0, \ldots$. 
We can visualize this structure in the spectrogram.

We will also compare the standard (linear-frequency) spectrogram with a **mel-spectrogram**, 
which uses a perceptually motivated frequency scale that better matches how humans hear pitch.

**Tasks:**
* Load the `trumpet` and `nutcracker` audio examples from librosa.
* Compute and display linear spectrograms and mel-spectrograms side by side.
* Use `n_fft=2048`, `hop_length=512`, and `n_mels=128`.

In [ ]:
# Load real audio
y_trumpet, sr_trumpet = librosa.load(librosa.example('trumpet'))
y_nutcracker, sr_nutcracker = librosa.load(librosa.example('nutcracker'))

print(f'Trumpet: {len(y_trumpet)/sr_trumpet:.1f}s at {sr_trumpet} Hz')
print(f'Nutcracker: {len(y_nutcracker)/sr_nutcracker:.1f}s at {sr_nutcracker} Hz')

# Play audio
print('\nTrumpet:')
ipd.display(ipd.Audio(y_trumpet, rate=sr_trumpet))
print('\nNutcracker:')
ipd.display(ipd.Audio(y_nutcracker, rate=sr_nutcracker))

In [ ]:
# Spectrograms: linear frequency vs mel scale
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# --- Trumpet ---
# TODO: Compute linear spectrogram of trumpet
# Hint: STFT, then take absolute value, then convert to dB
S_trumpet = # TODO
S_trumpet_db = # TODO

librosa.display.specshow(S_trumpet_db, sr=sr_trumpet, hop_length=512,
                         x_axis='time', y_axis='hz', ax=axes[0, 0])
axes[0, 0].set_title('Trumpet — Linear Spectrogram')
axes[0, 0].set_ylim([0, 8000])

# TODO: Compute mel spectrogram of trumpet
# Hint: librosa has a function that computes mel spectrograms directly
S_trumpet_mel = # TODO
S_trumpet_mel_db = # TODO

librosa.display.specshow(S_trumpet_mel_db, sr=sr_trumpet, hop_length=512,
                         x_axis='time', y_axis='mel', ax=axes[0, 1])
axes[0, 1].set_title('Trumpet — Mel Spectrogram')

# --- Nutcracker ---
# TODO: Repeat for nutcracker (linear and mel)
S_nut = # TODO
S_nut_db = # TODO

librosa.display.specshow(S_nut_db, sr=sr_nutcracker, hop_length=512,
                         x_axis='time', y_axis='hz', ax=axes[1, 0])
axes[1, 0].set_title('Nutcracker — Linear Spectrogram')
axes[1, 0].set_ylim([0, 8000])

S_nut_mel = # TODO
S_nut_mel_db = # TODO

librosa.display.specshow(S_nut_mel_db, sr=sr_nutcracker, hop_length=512,
                         x_axis='time', y_axis='mel', ax=axes[1, 1])
axes[1, 1].set_title('Nutcracker — Mel Spectrogram')

plt.tight_layout()
plt.show()

**Question:** Identify the fundamental frequency and harmonics in the trumpet signal. 
Why do musical signals show evenly spaced horizontal lines while broadband noise does not?

**Your answer here:**



## Task 2.4: Tone Removal — Filtering in the Frequency Domain

Now we put everything together in a practical application. We will:
1. Corrupt an audio signal by adding a loud pure tone (1 kHz sine wave).
2. Visualize the corruption in the spectrogram.
3. Design a **notch filter** in the STFT domain to remove the tone.
4. Listen to the before/after — the difference should be dramatic!

**Tasks:**
* Add a 1 kHz tone at amplitude 0.5 to the trumpet audio.
* Compute the STFT, identify the tone visually, and zero out the corresponding frequency bins.
* Reconstruct the audio via ISTFT and listen to the result.
* Experiment with different notch bandwidths.

In [ ]:
# Add a loud 1 kHz tone to the trumpet
tone_freq = 1000  # Hz
tone_amplitude = 0.5

# TODO: Generate the 1 kHz tone (same length as y_trumpet)
tone = # TODO

# TODO: Add tone to trumpet and normalize to prevent clipping
y_corrupted = # TODO
y_corrupted = y_corrupted / np.max(np.abs(y_corrupted))

print('Original trumpet:')
ipd.display(ipd.Audio(y_trumpet, rate=sr_trumpet))
print('\nCorrupted (with 1 kHz tone):')
ipd.display(ipd.Audio(y_corrupted, rate=sr_trumpet))

In [ ]:
# Visualize the corruption in the spectrogram
n_fft = 2048
hop_length = 512

S_orig = librosa.stft(y_trumpet, n_fft=n_fft, hop_length=hop_length)
S_corrupt = librosa.stft(y_corrupted, n_fft=n_fft, hop_length=hop_length)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

librosa.display.specshow(librosa.amplitude_to_db(np.abs(S_orig), ref=np.max),
                         sr=sr_trumpet, hop_length=hop_length,
                         x_axis='time', y_axis='hz', ax=axes[0])
axes[0].set_title('Original Trumpet')
axes[0].set_ylim([0, 6000])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(S_corrupt), ref=np.max),
                         sr=sr_trumpet, hop_length=hop_length,
                         x_axis='time', y_axis='hz', ax=axes[1])
axes[1].set_title('Corrupted (1 kHz tone visible as bright horizontal line)')
axes[1].set_ylim([0, 6000])

plt.tight_layout()
plt.show()

In [ ]:
# Design and apply a notch filter in the STFT domain
# Hint: get the frequency array for the STFT bins, then create a mask
# that is 1 everywhere except near the tone frequency.

# TODO: Get the frequency values for each STFT bin
freqs_stft = # TODO

# TODO: Create a notch mask — set bins near tone_freq to 0
notch_width = 50  # Hz on each side
notch_mask = # TODO

# TODO: Apply the mask to the corrupted STFT
# Hint: broadcast the 1D mask across all time frames
S_filtered = # TODO

# TODO: Reconstruct audio from filtered STFT
y_filtered = # TODO

print('Filtered audio (tone removed):')
ipd.display(ipd.Audio(y_filtered, rate=sr_trumpet))

In [ ]:
# Compare before/after spectrograms
S_filt_db = librosa.amplitude_to_db(np.abs(S_filtered), ref=np.max)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

librosa.display.specshow(librosa.amplitude_to_db(np.abs(S_orig), ref=np.max),
                         sr=sr_trumpet, hop_length=hop_length,
                         x_axis='time', y_axis='hz', ax=axes[0])
axes[0].set_title('Original (clean)')
axes[0].set_ylim([0, 6000])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(S_corrupt), ref=np.max),
                         sr=sr_trumpet, hop_length=hop_length,
                         x_axis='time', y_axis='hz', ax=axes[1])
axes[1].set_title('Corrupted (1 kHz tone)')
axes[1].set_ylim([0, 6000])

librosa.display.specshow(S_filt_db,
                         sr=sr_trumpet, hop_length=hop_length,
                         x_axis='time', y_axis='hz', ax=axes[2])
axes[2].set_title('After Notch Filter')
axes[2].set_ylim([0, 6000])

plt.suptitle('Tone Removal: Before and After', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Experiment with different notch widths
# Try narrow (±10 Hz), medium (±50 Hz), and wide (±200 Hz)
notch_widths = [10, 50, 200]

fig, axes = plt.subplots(1, len(notch_widths), figsize=(18, 5))

for i, nw in enumerate(notch_widths):
    # TODO: Create mask, filter, reconstruct, and display
    mask = # TODO
    
    S_filt = S_corrupt * mask[:, np.newaxis]
    y_filt = librosa.istft(S_filt, hop_length=hop_length, length=len(y_corrupted))
    
    S_filt_db = librosa.amplitude_to_db(np.abs(S_filt), ref=np.max)
    librosa.display.specshow(S_filt_db, sr=sr_trumpet, hop_length=hop_length,
                             x_axis='time', y_axis='hz', ax=axes[i])
    axes[i].set_title(f'Notch width = ±{nw} Hz')
    axes[i].set_ylim([0, 6000])
    
    print(f'Notch width ±{nw} Hz:')
    ipd.display(ipd.Audio(y_filt, rate=sr_trumpet))

plt.suptitle('Effect of Notch Filter Bandwidth', fontsize=14)
plt.tight_layout()
plt.show()

**Question:** What is the tradeoff when choosing the notch filter bandwidth? 
Describe what you hear when the notch is too narrow vs. too wide.

**Your answer here:**



## Task 2.5: Pitch Detection via Autocorrelation

Periodic signals have a powerful property: their **autocorrelation** peaks at multiples of the fundamental period. The autocorrelation of a discrete signal is:

$$ R[k] = \sum_{n} x[n] \, x[n+k] $$

For a periodic signal with fundamental period $T_0$ samples, $R[k]$ has its first prominent peak (after lag 0) at $k = T_0$. The fundamental frequency is then $f_0 = f_s / T_0$.

**Strategy:**
1. Extract a short frame (~200 ms) from the trumpet signal.
2. Compute the autocorrelation of that frame.
3. Find the first peak after a minimum lag (to avoid the trivial peak at lag 0 and implausibly high frequencies).
4. Convert the peak lag to a frequency estimate.
5. Extend to frame-by-frame pitch tracking and overlay on the spectrogram.

This is a real technique used in speech and music analysis — you are building a simplified pitch tracker from scratch.

In [ ]:
# Extract a single stable frame from the middle of the trumpet for development
# y_trumpet and sr_trumpet were loaded in Task 2.3

frame_duration = 0.2  # 200 ms — long enough to hear the pitch clearly
frame_len = int(sr_trumpet * frame_duration)
mid = len(y_trumpet) // 2
frame = y_trumpet[mid : mid + frame_len]

print(f'Frame: {frame_len} samples ({frame_duration*1000:.0f} ms) at {sr_trumpet} Hz')
print(f'Frequency resolution limit: {sr_trumpet / frame_len:.1f} Hz')

fig, ax = plt.subplots(figsize=(10, 3))
t_frame = np.arange(frame_len) / sr_trumpet * 1000
ax.plot(t_frame, frame)
ax.set_xlabel('Time [ms]')
ax.set_ylabel('Amplitude')
ax.set_title('Single Trumpet Frame (200 ms from middle of signal)')
ax.grid(True)
plt.tight_layout()
plt.show()

ipd.display(ipd.Audio(frame, rate=sr_trumpet))

In [ ]:
# Autocorrelation-based pitch detection on a single frame

# TODO: Compute the autocorrelation of `frame`
# Hint: use np.correlate(frame, frame, mode='full'), then take the right half
# (from the center onward) so that lag index 0 = zero lag
autocorr = # TODO

# TODO: Set a minimum lag to avoid detecting implausibly high frequencies
# For example, if max f0 is 1000 Hz, min_lag = sr_trumpet // 1000
min_lag = # TODO

# TODO: Find the first prominent peak in autocorr[min_lag:]
# Hint: you can use scipy.signal.find_peaks or simply np.argmax
# on the relevant slice, then add min_lag back to get the true lag
peak_lag = # TODO

# TODO: Convert lag to frequency
f0_estimate = # TODO

# Plot
fig, ax = plt.subplots(figsize=(10, 4))
lags_ms = np.arange(len(autocorr)) / sr_trumpet * 1000
ax.plot(lags_ms, autocorr)
ax.axvline(peak_lag / sr_trumpet * 1000, color='r', linestyle='--',
           label=f'Detected period = {peak_lag} samples → f0 = {f0_estimate:.1f} Hz')
ax.set_xlabel('Lag [ms]')
ax.set_ylabel('Autocorrelation')
ax.set_title('Autocorrelation of Trumpet Frame')
ax.legend()
ax.set_xlim([0, 30])  # zoom in to relevant range
ax.grid(True)
plt.tight_layout()
plt.show()

print(f'Estimated fundamental frequency: {f0_estimate:.1f} Hz')

In [ ]:
# Frame-by-frame pitch tracking over the full trumpet signal
n_fft_pitch = 2048
hop_pitch = 512
min_lag = sr_trumpet // 1000  # max f0 = 1000 Hz
max_lag = sr_trumpet // 50    # min f0 = 50 Hz

n_frames = 1 + (len(y_trumpet) - n_fft_pitch) // hop_pitch
f0_track = np.zeros(n_frames)

for i in range(n_frames):
    start = i * hop_pitch
    frame_i = y_trumpet[start : start + n_fft_pitch]
    
    # TODO: Compute autocorrelation of frame_i (same approach as above)
    autocorr_i = # TODO
    
    # TODO: Find first peak in autocorr_i[min_lag:max_lag]
    # and convert to frequency. Store in f0_track[i].
    # If no clear peak is found (e.g., silence), set f0_track[i] = 0.
    f0_track[i] = # TODO

# Overlay pitch track on spectrogram
S_trumpet_full = librosa.stft(y_trumpet, n_fft=n_fft_pitch, hop_length=hop_pitch)
S_trumpet_db = librosa.amplitude_to_db(np.abs(S_trumpet_full), ref=np.max)

fig, ax = plt.subplots(figsize=(14, 5))
librosa.display.specshow(S_trumpet_db, sr=sr_trumpet, hop_length=hop_pitch,
                         x_axis='time', y_axis='hz', ax=ax)
ax.set_ylim([0, 4000])

times = librosa.frames_to_time(np.arange(n_frames), sr=sr_trumpet, hop_length=hop_pitch)
valid = f0_track > 0
ax.plot(times[valid], f0_track[valid], 'r.', markersize=3, label='Detected f0')
ax.legend(loc='upper right')
ax.set_title('Pitch Tracking: Autocorrelation f0 Overlaid on Spectrogram')
plt.tight_layout()
plt.show()

**Question:** Is your pitch estimate consistent across frames? Where does it fail, and why? 
How could you make the detector more robust (e.g., parabolic interpolation around the peak, 
thresholding on autocorrelation peak height to reject unvoiced/silent frames)?

**Your answer here:**

